# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [2]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\L

## Load the dataset

In [3]:
dataset = 'imagenet' #'cifar10' #'imagenet'
splitsize = 100 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset= dataset, dataset_split=splitsize)

## Load and Prepare Adversarial Model

In [4]:
model = 'Salman2020Do_R18' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10, Salman2020Do_R18(Imagenet)
net, art_net, fb_net, alias = utils.get_model(dataset= dataset, modelname=model, norm='Linf') # specify the norm in case of robustbench models

c:\Users\d7000ko\Desktop\Python\elastic_net_adversarial_attack\.venv\Lib\site-packages\robustbench\utils.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = t

## Test Accuracy

In [5]:
utils.test_accuracy(net, xtest, ytest)
print(ytest)


Accuracy of the test set is: 49.000%

tensor([750, 889,   4, 873, 831, 644, 544, 217,  43, 636, 508, 670, 542, 438,
        803, 953, 914, 340, 875, 148, 902, 753, 296, 764, 617, 395, 280, 931,
        397, 786, 785, 164, 946, 306, 907, 416, 529, 514, 420, 988, 843, 984,
        505,   1, 137, 311, 222, 719, 759, 668, 615, 191, 704,  97, 160, 897,
         83, 388, 693, 586, 406, 354,  97, 412,  52, 934, 380, 113, 123, 954,
        158, 105, 857, 497, 407, 424, 644, 863, 885, 770, 804, 337, 246, 903,
        922, 270, 609, 403, 224, 874, 853, 358, 137, 667, 647,  57, 396, 875,
         62, 129])


## Parameters 

In [6]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations =100, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [7]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[1]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

		-------------- Hyperparameter Sweep for Attack: exp_attack_l1: learning_rate = 1 ----------------



KeyboardInterrupt: 

In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'original_AutoAttack_apgd-only',
                'exp_attack_l1'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)